# Abstract
The government of Bangalore has taken strict measures to contain the spreading of Covid-19. These measures have significantly reduced the number of vehicles on the road so as the amount of factory emissions. For this study, changes in the concentrations of six pollutants(CO, NO2, O3, PM10, PM2.5, and SO2) were evaluated duirng the lockdown period in the Central Silk Board area of Bangalore. 
On the basis of this quantitative analysis, we have reached the following conclusions:
1. Out of the six pollutants, five have significant difference after the lockdown in terms of concentration
2. There is an increase in air quality as AQI decreased by 15%

## Area of Interest
In this study we are focusing on the Central Silk Board junction in Bangalorem, India. This junction is known for its bottleneck traffic congestion as it is one of the busiest intersections in India. Lockdown began approximately 13 weeks into the year.
![](https://img.etimg.com/thumb/width-1200,height-900,imgsize-105406,resizemode-1,msid-65318241/news/politics-and-nation/four-lane-flyover-to-decongest-silk-board-junction-delayed.jpg)

In [ ]:
!pip install -q plotly==4.9.0
!pip install -q "notebook>=5.3" "ipywidgets>=7.2"
!pip install python-aqi

In [ ]:
# import packages
import os
import numpy as np #n-dimensional array operations
import scipy as sp #fundamental scientific computing
import pandas as pd #data manipulation and analysis
from matplotlib import pyplot as plt #plotting and visualization
import seaborn as sns #easy and beautiful data visualization
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from plotly.subplots import make_subplots
import aqi # to calculate AQI (Air Quality Index)
init_notebook_mode(connected=True)

## Effect of lockdown on air quality parameters

In [ ]:
# reading the datasets
co = pd.read_csv("../input/silkboard-bangalore-ambient-air-covid19lockdown/co_blr_aq_lockdown_effect_CSB.csv")
no2 = pd.read_csv("../input/silkboard-bangalore-ambient-air-covid19lockdown/no2_blr_aq_lockdown_effect_CSB.csv")
o3 = pd.read_csv("../input/silkboard-bangalore-ambient-air-covid19lockdown/o3_blr_aq_lockdown_effect_CSB.csv")
pm10 = pd.read_csv("../input/silkboard-bangalore-ambient-air-covid19lockdown/pm10_blr_aq_lockdown_effect_CSB.csv")
pm25 = pd.read_csv("../input/silkboard-bangalore-ambient-air-covid19lockdown/pm25_blr_aq_lockdown_effect_CSB.csv")
so2 = pd.read_csv("../input/silkboard-bangalore-ambient-air-covid19lockdown/so2_blr_aq_lockdown_effect_CSB.csv")
dfs = [co, no2, o3, pm10, pm25, so2]
names = ['co', 'no2', 'o3', 'pm10', 'pm25', 'so2']
for i, d in enumerate(dfs):
    d.name = names[i]

In [ ]:
# add column for week number to each dataset
for d in dfs:
    d.utc = pd.to_datetime(d.utc)
    d['week'] = d.utc.dt.week

In [ ]:
co.head()

In [ ]:
prev = pd.read_csv("../input/air-quality-data-in-india/city_day.csv")
prev.Date = pd.to_datetime(prev.Date)
prev['year'] = prev.Date.dt.year
prev = prev[prev.year==2018]
prev = prev[prev.City=="Bengaluru"]
prev['week']= prev.Date.dt.week
prev = prev.fillna(prev.mean())
prev = prev[prev.week > 6]
prev = prev[prev.week < 26]
prev = prev.rename(columns={'CO':names[0], 'NO2':names[1], 'O3':names[2], 'PM10':names[3], 'PM2.5':names[4], 'SO2':names[5]})
prev = prev.groupby('week').mean()
prev = prev.reset_index()
# co is measured in mg/m3 must be converted
prev.co = prev.co * 1000

In [ ]:
# create AQI (Air Quality Index) dataset
# we will be using the python-aqi library
aqidf = pd.DataFrame(co.utc)
for i, d in enumerate(dfs):
    aqidf[names[i]] = d.value
aqidf.utc = pd.to_datetime(d.utc)
aqidf['week'] = d.utc.dt.week

aqi_l = []
# we will approximate aqi using NO2, PM10, and PM25
for i in range(len(aqidf.utc)):
    aqi_l.append(aqi.to_aqi([
        (aqi.POLLUTANT_NO2_1H, str(aqidf.no2[i])),
        (aqi.POLLUTANT_PM10, str(aqidf.pm10[i])),
        (aqi.POLLUTANT_PM25, str(aqidf.pm25[i])),
    ]))
aqi_col = pd.Series(aqi_l)

aqidf['aqi'] = aqi_col
aqidf.aqi = aqidf.aqi.astype('int64')
aqidf = aqidf.dropna()
aqidf.week = aqidf.week.astype('int64')

### CO, NO2, O3, PM10, PM25, SO2 concentrations over time

In [ ]:
f, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
rows, cols = 2, 3
n = 0
for row in range(rows):
    for col in range(cols):
        #plot this year
        sns.lineplot(x=dfs[n].week, y=dfs[n].value, ax=axes[row,col])
        # set unit as label
        axes[row, col].set_ylabel(str(dfs[n].unit[0]))
        # plot previous year
        axes[row, col].plot(prev.week, prev[names[n]], color='orange', linestyle="-.", label='2018')
        # perform regression
        z = np.polyfit(dfs[n].week, dfs[n].value, 1)
        p = np.poly1d(z)
        # plot regression line
        axes[row, col].plot(dfs[n].week, p(dfs[n].week), "r--", label=str(p))
        # lockdown line
        axes[row, col].axvline(13, color='darkred', label='lockdown')
        axes[row, col].set_title(dfs[n].name)
        axes[row, col].grid(linestyle='--')
        axes[row, col].legend()
        n+=1

plt.show()

In [ ]:
f, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
rows, cols = 2, 3
n = 0
for row in range(rows):
    for col in range(cols):
        # plot this year
        sns.lineplot(x=dfs[n].week, y=dfs[n].value, ax=axes[row,col])
        # set unit as label
        axes[row, col].set_ylabel(str(dfs[n].unit[0]))
        # mean before lockdown
        axes[row, col].hlines(dfs[n][dfs[n].week<=13].value.mean(), 7.5, 13, color='darkred', linestyle='dashed', label='Mean before lockdown')
        # mean after lockdown
        axes[row, col].hlines(dfs[n][dfs[n].week>13].value.mean(), 13, 25, color='darkred', linestyle='dashed', label='Mean after lockdown')
        # lockdown line
        axes[row, col].axvline(13, color='darkred', label='lockdown')
        axes[row, col].set_title(dfs[n].name)
        axes[row, col].grid(linestyle='--')
        axes[row, col].legend()
        n+=1

plt.show()

Changes are obvious in all pollutants except for O3

### AQI (Air Quality Index) Over Time

In [ ]:
sns.lineplot(x=aqidf.week, y=aqidf.aqi, label='AQI')
z = np.polyfit(aqidf.week, aqidf.aqi, 1)
p = np.poly1d(z)
plt.plot(aqidf.week, p(aqidf.week), "r--", label=str(p))
plt.plot(prev.week, prev.AQI, color='orange', label='2018')
plt.axhspan(100, 150, facecolor='orange', alpha=0.2, label='Unhealthy')
plt.axhspan(51, 100, facecolor='yellow', alpha=0.2, label='Moderate')
plt.axhspan(0, 50, facecolor='green', alpha=0.2, label='Good')
plt.axvline(13, color='darkred', label='lockdown')
plt.legend()
plt.box(False)
plt.grid(linestyle='--')
plt.show()


In [ ]:
aqi_before = aqidf[aqidf.week<=13].aqi.mean()
aqi_after = aqidf[aqidf.week>13].aqi.mean()
sns.lineplot(x=aqidf.week, y=aqidf.aqi, label='AQI')
plt.axhspan(100, 150, facecolor='orange', alpha=0.2, label='Unhealthy')
plt.axhspan(51, 100, facecolor='yellow', alpha=0.2, label='Moderate')
plt.axhspan(0, 50, facecolor='green', alpha=0.2, label='Good')
plt.axvline(13, color='darkred', label='lockdown')
plt.hlines(aqi_before, 7.5, 13, color='darkred', linestyle='dashed', label='MAQI before lockdown')
plt.hlines(aqi_after, 13, 25, color='darkred', linestyle='dashed', label='MAQI after lockdown')
plt.legend()
plt.box(False)
plt.grid(linestyle='--')
plt.show()


In [ ]:
print("aqi before: " + str(aqi_before))
print("aqi after: " + str(aqi_after))
100 * (aqi_after - aqi_before)/aqi_before

# Conclusion and Discussion
There were significant differences(p<0.01) in the AQI and the concentrations of five(CO, NO2, PM10, PM25, SO2) out of the six pollutans evaluated. Suggesting human activity and mobility are closely correlated to air quality. The AQI decreased by 15.427%, from 81.258 to 68.722 before and after lockdown. 